In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
# from torch_geometric.data import Data
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis
from sklearn.preprocessing import StandardScaler

# Load the data from the CSV file
df = pd.read_csv('sensor.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220320 entries, 0 to 220319
Data columns (total 55 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      220320 non-null  int64  
 1   timestamp       220320 non-null  object 
 2   sensor_00       210112 non-null  float64
 3   sensor_01       219951 non-null  float64
 4   sensor_02       220301 non-null  float64
 5   sensor_03       220301 non-null  float64
 6   sensor_04       220301 non-null  float64
 7   sensor_05       220301 non-null  float64
 8   sensor_06       215522 non-null  float64
 9   sensor_07       214869 non-null  float64
 10  sensor_08       215213 non-null  float64
 11  sensor_09       215725 non-null  float64
 12  sensor_10       220301 non-null  float64
 13  sensor_11       220301 non-null  float64
 14  sensor_12       220301 non-null  float64
 15  sensor_13       220301 non-null  float64
 16  sensor_14       220299 non-null  float64
 17  sensor_15 

In [5]:
df=df.drop("sensor_15",axis=1)

# Drop duplicates
df = df.drop_duplicates()

df = df.fillna(df.mean())

/var/folders/4_/43c1czgs17s2s8z18wlcyc540000gn/T/ipykernel_6139/976719905.py:6: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220320 entries, 0 to 220319
Data columns (total 54 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      220320 non-null  int64  
 1   timestamp       220320 non-null  object 
 2   sensor_00       220320 non-null  float64
 3   sensor_01       220320 non-null  float64
 4   sensor_02       220320 non-null  float64
 5   sensor_03       220320 non-null  float64
 6   sensor_04       220320 non-null  float64
 7   sensor_05       220320 non-null  float64
 8   sensor_06       220320 non-null  float64
 9   sensor_07       220320 non-null  float64
 10  sensor_08       220320 non-null  float64
 11  sensor_09       220320 non-null  float64
 12  sensor_10       220320 non-null  float64
 13  sensor_11       220320 non-null  float64
 14  sensor_12       220320 non-null  float64
 15  sensor_13       220320 non-null  float64
 16  sensor_14       220320 non-null  float64
 17  sensor_16 

In [7]:
# Define the mapping of machine_status categories to numeric labels
label_mapping = {'NORMAL': 0, 'BROKEN': 1, 'RECOVERING': 2}

# Convert machine_status column to numeric labels
df['label'] = df['machine_status'].map(label_mapping)

# Calculate the mean of each sensor reading
# sensor_columns = df.columns[2:-1]  # Exclude the timestamp and machine_status columns
# mean_values = df[sensor_columns].mean()

# Let's convert the data type of timestamp column to datetime format
df['date'] = pd.to_datetime(df['timestamp'])
del df['timestamp']

df_d=df.drop(["date","Unnamed: 0","machine_status"],axis=1)

# Select only columns with numeric data types
# X = df.select_dtypes(include=[float])

In [8]:
df_d.columns

Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
       'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
       'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
       'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
       'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25',
       'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30',
       'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35',
       'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40',
       'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45',
       'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor_50',
       'sensor_51', 'label'],
      dtype='object')

In [9]:


# Split the data into training, validation, and testing subsets
train_end_index = int(len(df_d) * 0.5)  # 50% for training
valid_end_index = int(len(df_d) * 0.9)  # 40% for validation, remaining for testing

train_data = df_d[:train_end_index]
valid_data = df_d[train_end_index:valid_end_index]
test_data = df_d[valid_end_index:]


In [10]:
len(train_data), len(valid_data), len(test_data)

(110160, 88128, 22032)

In [11]:
y_train=train_data['label'].values
y_valid=valid_data['label'].values
y_test=test_data['label'].values

train_data.drop("label",axis=1,inplace=True)
valid_data.drop("label",axis=1,inplace=True)
test_data.drop("label",axis=1,inplace=True)
# train_labels = train_labels.long()
# Convert the training, validation, and testing dataframes to tensors


# Preprocess the data
scaler = StandardScaler()
x_train = scaler.fit_transform(train_data)
x_valid = scaler.transform(valid_data)
x_test = scaler.transform(test_data)

# Convert the preprocessed data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
x_valid = torch.tensor(x_valid, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)

y_train=torch.tensor(y_train,dtype=torch.long)
y_valid=torch.tensor(y_valid,dtype=torch.long)
y_test=torch.tensor(y_test,dtype=torch.long)


/var/folders/4_/43c1czgs17s2s8z18wlcyc540000gn/T/ipykernel_6139/1161557762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop("label",axis=1,inplace=True)
/var/folders/4_/43c1czgs17s2s8z18wlcyc540000gn/T/ipykernel_6139/1161557762.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data.drop("label",axis=1,inplace=True)
/var/folders/4_/43c1czgs17s2s8z18wlcyc540000gn/T/ipykernel_6139/1161557762.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [12]:
x_train.shape

torch.Size([110160, 51])

In [13]:
df_d['label'].value_counts()

0    205836
2     14477
1         7
Name: label, dtype: int64

In [14]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as data 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_size =  x_train.shape[1]
# hidden_size = 100
hidden_size1 = 64
hidden_size2 = 64
num_classes = 3
num_epochs = 500
batch_size = 100
learning_rate = 0.01



train_dataset =data.TensorDataset(x_train, y_train)
train_loader =data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset =data.TensorDataset(x_valid, y_valid)
valid_loader =data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

test_dataset =data.TensorDataset(x_test, y_test)
test_loader =data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.l2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.l3 = nn.Linear(hidden_size2, num_classes)
        self.dropout1=nn.Dropout(0.5)
        self.dropout2=nn.Dropout(0.5)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.l2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        out = self.l3(out)
        #out = self.softmax(out)
        return out

model = NeuralNet(input_size, hidden_size1, hidden_size2, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    total_loss=0
    model.train()
    for i, (X_batch, sample_labels) in enumerate(train_loader):
        (X_batch,sample_labels)=X_batch.to(device),sample_labels.to(device)
        
       
        # Forward pass
        outputs = model(X_batch)
        # print(outputs)
        loss = criterion(outputs, sample_labels)


        # Backward and optimize
        # optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        #accumlate loss
        total_loss +=loss.item()

    # Print the training loss for each epoch
    if (epoch + 1) % 50 ==0 :
       print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader)}")


       


Epoch [50/500], Loss: 0.0118417413493551
Epoch [100/500], Loss: 0.007616562328341165
Epoch [150/500], Loss: 0.009782342277927097
Epoch [200/500], Loss: 0.028477835686544457
Epoch [250/500], Loss: 0.03827745568417182
Epoch [300/500], Loss: 0.008702781128359684
Epoch [350/500], Loss: 0.023818801225044824
Epoch [400/500], Loss: 0.014002390148992575
Epoch [450/500], Loss: 0.009708657409004569
Epoch [500/500], Loss: 0.004186744340487731


In [98]:
# Test the model
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for X_batch, labels in test_loader:
        X_batch = X_batch.to(device)
        labels = labels.to(device)
        outputs = model(X_batch)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test set: {acc} %')

Accuracy of the network on the test set: 99.93645606390704 %


In [15]:
from sklearn.metrics import f1_score, balanced_accuracy_score, recall_score, precision_score, confusion_matrix

# 5. Model Evaluation

model.eval()  # set the model to evaluation mode

y_true = []
y_pred = []

with torch.no_grad():
    for data, label in valid_loader:
        data, label = data.to(device), label.to(device)
        
        outputs = model(data)
        
        # Convert outputs to predicted classes
        _, predicted = torch.max(outputs.data, 1)
        
        y_true.extend(label.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Compute the metrics
f1 = f1_score(y_true, y_pred, average='macro')
bal_accuracy = balanced_accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')

print(f'F1 Score: {f1}')
print(f'Balanced Accuracy: {bal_accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')


F1 Score: 0.6380771577683327
Balanced Accuracy: 0.6338921582293372
Recall: 0.6338921582293372
Precision: 0.6424144704144154


/Users/mahmoud/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
